# Foodle Maps

This notebook gives practical examples of the potentiality of Foodle Maps.
First, load the necessary packages to perform SPARQL queries:

In [50]:
from SPARQLWrapper import SPARQLWrapper, JSON, DIGEST, POST

sparql = SPARQLWrapper("https://sparql.alexmv12.xyz/FoodleMaps")

Then, we can start with a simple query: show all the restaurants in the KB.

In [2]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    
    SELECT ?restaurant 
    WHERE { ?restaurant a schema:FoodEstablishment  }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"])

https://alexmv12.github.io/FoodleMaps/La_Cantinetta_caprese
https://alexmv12.github.io/FoodleMaps/Tavernacolo
https://alexmv12.github.io/FoodleMaps/Trattoria_Sole_Milano
https://alexmv12.github.io/FoodleMaps/#Trattoria_dell_orsa


... or, we want to see some of the available dishes.

In [8]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    
    SELECT ?dish 
    WHERE { ?dish a schema:MenuItem  }
    LIMIT 5
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["dish"]["value"])

https://alexmv12.github.io/FoodleMaps/#Pasta_alla_carbonara
https://alexmv12.github.io/FoodleMaps/#Baba_al_rum
https://alexmv12.github.io/FoodleMaps/#Bruschettone_gorgonzola_noci_e_salame
https://alexmv12.github.io/FoodleMaps/#Caprese_di_bufala
https://alexmv12.github.io/FoodleMaps/#Cotoletta_alla_bolognese


## Examples

- Show me all the restaurants serving dishes with tomatoes.
- Find the nearest restaurant which serves at least a “light” dish.
- Show me a restaurant which does not serve any dish with eggs.
- Show me a restaurant which serves a dish which can be prepared in less than 5 minutes.

### Dishes with tomatoes
Let's find all restaurants which serve dishes which contain fresh tomatoes.

In [10]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT ?restaurant
    WHERE { ?restaurant schema:hasMenu ?menu.
            ?menu foodlemaps:servesRecipe ?recipe.
            ?recipe foodlemaps:hasIngredient foodlemaps:Tomato .}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"])

https://alexmv12.github.io/FoodleMaps/#La_Cantinetta_caprese
https://alexmv12.github.io/FoodleMaps/#Tavernacolo
https://alexmv12.github.io/FoodleMaps/#Trattoria_Sole_Milano


### Fresh dishes for summer

Or, maybe we want a "fresh" dish for this hot summer. What are some dishes with that property?

In [11]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT ?dish
    WHERE { ?dish foodlemaps:hasCharacteristic foodlemaps:Fresh}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["dish"]["value"])

https://alexmv12.github.io/FoodleMaps/#Insalata_greca
https://alexmv12.github.io/FoodleMaps/#Vitello_tonnato


Great! Which restaurants serve "Insalata greca"?

In [13]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT ?restaurant
    WHERE { ?restaurant schema:hasMenu ?menu.
            ?menu foodlemaps:servesRecipe ?recipe.
            foodlemaps:Insalata_greca foodlemaps:hasRecipe ?recipe .}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"])

https://alexmv12.github.io/FoodleMaps/#Tavernacolo


We'd like the address and telephone number of "Tavernacolo", so we could book a table.

In [32]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT ?address ?telephoneNumber
    WHERE { foodlemaps:Tavernacolo schema:address ?address ;
        			               schema:telephone ?telephoneNumber .
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["address"]["value"])
    print(result["telephoneNumber"]["value"])

Via Bassini, 45, Milano, Italy
02 1234 4321


### Eggs-free restaurants
We want to find restaurants which do not serve eggs at all. Maybe we are allergic, so we prefere to not risk any contamination.

In [40]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT DISTINCT ?restaurant
    WHERE { 
            ?restaurant a schema:FoodEstablishment
            
            FILTER NOT EXISTS { 
                ?restaurant schema:hasMenu ?menu.
                ?menu foodlemaps:servesRecipe ?recipe.
                ?recipe foodlemaps:hasIngredient foodlemaps:Egg 
            }
    }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"])

https://alexmv12.github.io/FoodleMaps/#Tavernacolo


As a counter-proof, we can list all the restaurants which serve dishes with eggs:

In [43]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT ?restaurant ?menuItem
    WHERE { ?restaurant schema:hasMenu ?menu.
            ?menu foodlemaps:servesRecipe ?recipe.
            ?recipe foodlemaps:hasIngredient foodlemaps:Egg .
            ?menuItem foodlemaps:hasRecipe ?recipe .
    }

""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"] + " -> " + result["menuItem"]["value"])

https://alexmv12.github.io/FoodleMaps/#La_Cantinetta_caprese -> https://alexmv12.github.io/FoodleMaps/#Baba_al_rum
https://alexmv12.github.io/FoodleMaps/#Trattoria_dell_orsa -> https://alexmv12.github.io/FoodleMaps/#Cotoletta_alla_bolognese
https://alexmv12.github.io/FoodleMaps/#Trattoria_dell_orsa -> https://alexmv12.github.io/FoodleMaps/#Tortellini_in_brodo
https://alexmv12.github.io/FoodleMaps/#Trattoria_Sole_Milano -> https://alexmv12.github.io/FoodleMaps/#Pasta_alla_carbonara
https://alexmv12.github.io/FoodleMaps/#Trattoria_Sole_Milano -> https://alexmv12.github.io/FoodleMaps/#Tiramisu
https://alexmv12.github.io/FoodleMaps/#La_Cantinetta_caprese -> https://alexmv12.github.io/FoodleMaps/#Vitello_tonnato


## Fast restaurants
We are in a hurry! We need a restaurant which can serve us a very fast dish. Let's see some of the fastest restaurants in our ontology.

In [70]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT ?restaurant ?menuItem ?preparationTime
    WHERE { ?restaurant schema:hasMenu ?menu.
            ?menu foodlemaps:servesRecipe ?recipe.
            ?recipe foodlemaps:hasIngredient foodlemaps:Egg .
            ?menuItem foodlemaps:hasRecipe ?recipe .
            ?menuItem foodlemaps:hasPreparationTime ?preparationTime .
            FILTER (?preparationTime < 10)
    }

""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"] + " can serve " + result["menuItem"]["value"] + " in " + result["preparationTime"]["value"] + " minutes.")

https://alexmv12.github.io/FoodleMaps/#La_Cantinetta_caprese can serve https://alexmv12.github.io/FoodleMaps/#Baba_al_rum in 3 minutes.
https://alexmv12.github.io/FoodleMaps/#La_Cantinetta_caprese can serve https://alexmv12.github.io/FoodleMaps/#Vitello_tonnato in 6 minutes.


## Extra: user preferences
The ontology also includes a set of classes which can be used to represent the preferences of a user.  
For example, we may create a user which follows a vegetarian diet:

In [107]:
sparql.setHTTPAuth(DIGEST)
sparql.setCredentials("admin", "insert_sparql_endpoint_password_here")
sparql.setMethod(POST)

sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    INSERT DATA
    { 
      foodlemaps:user0                       a foodlemaps:User ;
                       foodlemaps:requiresDiet schema:VegetarianDiet .
    }
""")

results = sparql.query()
print(results.response.read())

b'<html>\n<head>\n</head>\n<body>\n<h1>Success</h1>\n<p>\nUpdate succeeded\n</p>\n</body>\n</html>\n'


What are some of the dishes we can eat?

In [110]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX foodlemaps: <https://alexmv12.github.io/FoodleMaps/#>
    
    SELECT ?restaurant ?menuItem
    WHERE { 
            foodlemaps:user0 foodlemaps:requiresDiet ?diet .
            
            ?restaurant schema:hasMenu ?menu .
            ?menu foodlemaps:servesRecipe ?recipe.
            ?menuItem foodlemaps:hasRecipe ?recipe .
            ?recipe schema:suitableForDiet ?diet .            
    }

""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"] + " -> " + result["menuItem"]["value"] )

https://alexmv12.github.io/FoodleMaps/#La_Cantinetta_caprese -> https://alexmv12.github.io/FoodleMaps/#Baba_al_rum
https://alexmv12.github.io/FoodleMaps/#Tavernacolo -> https://alexmv12.github.io/FoodleMaps/#Insalata_greca
